In [1]:
from sklearn.svm import SVC
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn import datasets
from sklearn.base import clone

In [2]:
iris = datasets.load_iris()

# Funcao abaixo testada

# Pré-processamento 
(em construção)

In [3]:
def pre_processing():
    return spike.data, spike.target
def binShuffle(data):
    return data
def trialShuffle(data):
    return data

# Classe decodificadora
(Está nos planos separar em duas classes pra fazer mais sentido)

In [24]:
class Animal():
    def __init__(self, animalData, trainSize, Nclf_ForEachTrial):
        self.trialsToUse = range(150)
        self.classifiers = {itrial:[] for itrial in self.trialsToUse}
        self.predictions = {itrial:[] for itrial in self.trialsToUse}
        self.trainSize = trainSize
        self.bestClassifier = None
        self.Nclf_ForEachTrial = Nclf_ForEachTrial
        self.Data = animalData

    def __generateClassifier(self,testTrial):
        trainIndexes = self.__getTrainIndexes(testTrial)
        clf = self.__trainOneClassifier(trainIndexes)
        self.__addClassifierToAll(trainIndexes, clf)

    def __addClassifierToAll(self,trainIndexes, clf):
        possibleTestTrials = list( set(self.trialsToUse) - set(trainIndexes) )
        for iTestTrial in possibleTestTrials:
            if len(self.classifiers[iTestTrial]) < self.Nclf_ForEachTrial:
                self.classifiers[iTestTrial].append(clf)

    def __trainOneClassifier(self,trainIndexes):
        if self.bestClassifier is None:
            self.__gridSearch()
        clf = clone(self.bestClassifier)
        X, y = self.__getTrialsData(trainIndexes)
        clf.fit(X,y)
        return clf

    def __gridSearch(self):
        grid_trials = self.trialsToUse
        X, y = self.__getTrialsData(grid_trials)
        clf = SVC( kernel='rbf' )
        grid = GridSearchCV(clf, self.__gridParams(), cv=5)
        grid.fit(X, y)
        self.bestClassifier = grid.best_estimator_

    def __gridParams(self):
        C_grid = range(1,17);
        Gamma_grid = np.linspace(0.0156,0.25,20)
        return {'C':C_grid,'gamma':Gamma_grid}


    def __getTrainIndexes(self,testTrial):
        nonTestTrials = self.trialsToUse[:]
        nonTestTrials.remove(testTrial)
        trainIndexes = np.random.choice(nonTestTrials, self.trainSize, replace=False)
        return trainIndexes

    def __getTrialsData(self,trialIndexes):
        #função definida dessa forma temporariamente, só para testes
        X = iris.data[trialIndexes]
        y = iris.target[trialIndexes]
        return X, y

    def generateClassifiers(self):
        for iTestTrial in self.trialsToUse:
            while len(self.classifiers[iTestTrial]) < self.Nclf_ForEachTrial:
                self.__generateClassifier(iTestTrial)


    def predictAllResults(self):
        for iTestTrial in self.trialsToUse:
            for iclf in self.classifiers[iTestTrial]:
                X, y = self.__getTrialsData(iTestTrial)
                self.predictions[iTestTrial].append(iclf.predict(X))
            #self.pearsonTest(iTestTrial)

    def pearsonTest(self,iTestTrial):
        self.predictions['corr'+str(iTestTrial)] = [scipy.stats.pearsonr(yp,y) for yp in self.predictions[iTestTrial] ]

Os erros abaixo são devido ao iris_dataset só ter 1 elemento em cada trial. A função responsável _getTrialsData_ vai ser mudada completamente pra lidar com os dados em novo formato.

In [25]:
rat = Animal([], 30, 10)
rat.generateClassifiers()
rat.predictAllResults()

/home/estevao/py2_kernel/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/estevao/py2_kernel/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/estevao/py2_kernel/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) i

In [30]:
rat.classifiers[0] #temos 10 classificadores para cada trial,
                   # devido ao terceiro parametro de Animal([], 30, 10)
                   # No caso dos dados geraremos 30, como o paper

[SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape=None, degree=3, gamma=0.18831578947368421,
   kernel='rbf', max_iter=-1, probability=False, random_state=None,
   shrinking=True, tol=0.001, verbose=False),
 SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape=None, degree=3, gamma=0.18831578947368421,
   kernel='rbf', max_iter=-1, probability=False, random_state=None,
   shrinking=True, tol=0.001, verbose=False),
 SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape=None, degree=3, gamma=0.18831578947368421,
   kernel='rbf', max_iter=-1, probability=False, random_state=None,
   shrinking=True, tol=0.001, verbose=False),
 SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape=None, degree=3, gamma=0.18831578947368421,
   kernel='rbf', max_iter=-1, probability=False, random_state=None,
   shrinking=True, tol=0.001, verbose=False),
 SVC(C=2, cache_size=200, class_weight=N

In [43]:
print('Correta: '+str(iris.target[15]))
print('Previsao dos classificadores')
print(rat.predictions[15])

print('\nCorreta: '+str(iris.target[60]))
print('Previsao dos classificadores')
print(rat.predictions[60])

print('\nCorreta: '+str(iris.target[115]))
print('Previsao dos classificadores')
print(rat.predictions[130])

Correta: 0
Previsao dos classificadores
[array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0])]

Correta: 1
Previsao dos classificadores
[array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1])]

Correta: 2
Previsao dos classificadores
[array([2]), array([2]), array([2]), array([2]), array([2]), array([2]), array([2]), array([2]), array([2]), array([2])]
